In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/src/sensorcnn")

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
import pandas as pd
import matplotlib
matplotlib.style.use('ggplot')

In [3]:
from visualization.examples import *
from dataset.setup_exercise_examples import *
from dataset.utils import *

In [4]:
all_samples = {}
for filename in csv_file_iterator("/data/Setup Data/Mo"):
    samples = load_from_csv(filename)
    all_samples[filename] = samples

In [5]:
sample1 = all_samples['/data/Setup Data/Mo/2016-05-24T22-36-02Z.csv']

In [6]:
from functools import reduce
df = reduce(lambda x, y: x.append(y), all_samples.values())

In [7]:
df['label'].value_counts()

no_exercise                                                   28987
resistanceTargeted:shoulders/dumbbell-shoulder-press           6859
resistanceTargeted:arms/triceps-extension                      6673
resistanceTargeted:shoulders/lateral-raise                     5346
resistanceTargeted:arms/dumbbell-biceps-curl                   5118
setup_resistanceTargeted:arms/triceps-extension                2750
setup_resistanceTargeted:shoulders/dumbbell-shoulder-press     2500
setup_resistanceTargeted:arms/dumbbell-biceps-curl             2500
setup_resistanceTargeted:shoulders/lateral-raise               2250
Name: label, dtype: int64

In [8]:
labels = [l for l in df['label'].value_counts().keys() if l.startswith("setup_")]
labels_map = {l:i+1 for (i,l) in enumerate(labels)}
label_to_idx = lambda l: labels_map.get(l, 0)

In [9]:
df['target'] = df['label'].map(label_to_idx)

In [10]:
samples = df[['x', 'y', 'z']].as_matrix()

In [11]:
positive_targets = df[df['target'] > 0]
# negative_targets = df[df['target'] == 0]
# targets = np.append(positive_targets[['target']].as_matrix(),
#                     negative_targets[['target']].as_matrix(),
#                     axis=0)

In [12]:
# User positive only
targets = positive_targets[['target']].as_matrix()
samples = df[['x', 'y', 'z']].as_matrix()[positive_targets.index]

In [13]:
df['label'].map(label_to_idx).value_counts()

0    52983
1     2750
3     2500
2     2500
4     2250
Name: label, dtype: int64

In [14]:
print(samples.shape)
print(targets.shape)

(10000, 3)
(10000, 1)


In [15]:
X, y = augment_examples(samples, targets, new_sample_size=50*5)

In [16]:
print(X.shape)
print(y.shape)

(40, 750)
(40, 1)


### Examples encoding

In [17]:
Y = np.zeros((y.shape[0], 4))
iy = np.array([[i, yi-1] for (i, yi) in enumerate(y) if yi > 0])
Y[iy[:, 0], iy[:, 1]] = 1

### Split dataset

In [19]:
X_train, X_test, Y_train, Y_test = split(X, Y)

In [37]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(input_dim=X.shape[1], output_dim=150, activation="tanh", init='uniform'))
model.add(Dropout(0.5))
model.add(Dense(input_dim=150, output_dim=100, activation="tanh", init='uniform'))
model.add(Dropout(0.5))
model.add(Dense(input_dim=100, output_dim=4, activation="tanh", init='uniform'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=["accuracy"])

In [38]:
model.fit(X_train, Y_train, nb_epoch=30, batch_size=128)

Epoch 1/30
28/28 [==============================] - 0s - loss: 0.2561 - acc: 0.0357
Epoch 2/30
28/28 [==============================] - 0s - loss: 0.2282 - acc: 0.2500
Epoch 3/30
28/28 [==============================] - 0s - loss: 0.2120 - acc: 0.2143
Epoch 4/30
28/28 [==============================] - 0s - loss: 0.2030 - acc: 0.2857
Epoch 5/30
28/28 [==============================] - 0s - loss: 0.1837 - acc: 0.4286
Epoch 6/30
28/28 [==============================] - 0s - loss: 0.1854 - acc: 0.3929
Epoch 7/30
28/28 [==============================] - 0s - loss: 0.1890 - acc: 0.4286
Epoch 8/30
28/28 [==============================] - 0s - loss: 0.1788 - acc: 0.5357
Epoch 9/30
28/28 [==============================] - 0s - loss: 0.1842 - acc: 0.4643
Epoch 10/30
28/28 [==============================] - 0s - loss: 0.1787 - acc: 0.5357
Epoch 11/30
28/28 [==============================] - 0s - loss: 0.1785 - acc: 0.4643
Epoch 12/30
28/28 [==============================] - 0s - loss: 0.1693 - a

In [39]:
model.predict(X_test)

array([[-0.02799169,  0.18997015,  0.64517057,  0.10524564],
       [-0.22060394,  0.64599442,  0.42282325,  0.00146477],
       [ 0.61788076,  0.15217321,  0.31897968, -0.01920284],
       [ 0.10421221,  0.26466757,  0.35425496,  0.34758511],
       [-0.47468522,  0.56679517,  0.18433185,  0.63491309],
       [-0.05861086,  0.31878594,  0.44696212,  0.3030211 ],
       [ 0.08347926,  0.22002435,  0.48012936,  0.20979737],
       [ 0.29429615,  0.19146495,  0.33237538,  0.30819041],
       [ 0.30576468,  0.2259592 ,  0.17905208,  0.38010305],
       [ 0.08367258,  0.34312558,  0.24523543,  0.36538488],
       [ 0.16005066,  0.29212341,  0.28321302,  0.35127568],
       [-0.06104065,  0.09799861,  0.35907009,  0.57037264]])

In [45]:
class X(object):
    def __init__(self):
        self.x = 8

x = X()
def f(x):
    return x.x
X.f = f
print(x.f())

8


In [46]:
9 + 1

10

In [50]:
In

['',
 'get_ipython().magic(\'pylab inline\')\nget_ipython().magic(\'load_ext autoreload\')\nget_ipython().magic(\'autoreload 2\')\nimport sys\nsys.path.append("/src/sensorcnn")',
 "import sys\nimport pandas as pd\nimport matplotlib\nmatplotlib.style.use('ggplot')",
 'from visualization.examples import *\nfrom dataset.setup_exercise_examples import *\nfrom dataset.utils import *',
 'all_samples = {}\nfor filename in csv_file_iterator("/data/Setup Data/Mo"):\n    samples = load_from_csv(filename)\n    all_samples[filename] = samples',
 "sample1 = all_samples['/data/Setup Data/Mo/2016-05-24T22-36-02Z.csv']",
 'from functools import reduce\ndf = reduce(lambda x, y: x.append(y), all_samples.values())',
 "df['label'].value_counts()",
 'labels = [l for l in df[\'label\'].value_counts().keys() if l.startswith("setup_")]\nlabels_map = {l:i+1 for (i,l) in enumerate(labels)}\nlabel_to_idx = lambda l: labels_map.get(l, 0)',
 "df['target'] = df['label'].map(label_to_idx)",
 "samples = df[['x', 'y',